In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

In [84]:
# Read the csv file into a pandas DataFrame
# perfs = pd.read_csv('Perf_OrFracs.csv', encoding = "utf-8")
# perfs.head()

In [3]:
perfs = pd.read_excel('../FinalProject/WellInfo_Cleaned.xlsx', sheet_name='Perfs_Productions') 
perfs.head()

,Well,MD,Perfed,Fracced,Lithology,Age,API_PVT,API_CUPET,API_Avg,DIFF_API,...,CurrentRate,CurrentWC,WaterRate,CumOil,MaxOil,BeforeFrac,Normalised,FracSize,BeforeFracProd,AfterFracProd
0,P-103,753,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
1,P-103,754,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
2,P-103,755,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
3,P-103,756,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
4,P-103,757,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0


In [4]:
perfs.head()

,Well,MD,Perfed,Fracced,Lithology,Age,API_PVT,API_CUPET,API_Avg,DIFF_API,...,CurrentRate,CurrentWC,WaterRate,CumOil,MaxOil,BeforeFrac,Normalised,FracSize,BeforeFracProd,AfterFracProd
0,P-103,753,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
1,P-103,754,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
2,P-103,755,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
3,P-103,756,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
4,P-103,757,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.0,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0


In [5]:
data = perfs
convert_dict = {'Age': str, 
                'Lithology': str
               } 


data = data.astype(convert_dict) 
print(data.dtypes) 



# data = data["Lithology"].astype(str)
data

Well               object
MD                  int64
Perfed              int64
Fracced             int64
Lithology          object
Age                object
API_PVT           float64
API_CUPET         float64
API_Avg           float64
DIFF_API          float64
GasGravity        float64
OilGravity        float64
P_Sat             float64
Uo                float64
Ug                float64
Bo                float64
Bg                float64
CO2               float64
H2S               float64
InitialRate       float64
CurrentRate       float64
CurrentWC         float64
WaterRate         float64
CumOil            float64
MaxOil            float64
BeforeFrac        float64
Normalised        float64
FracSize          float64
BeforeFracProd    float64
AfterFracProd     float64
dtype: object


,Well,MD,Perfed,Fracced,Lithology,Age,API_PVT,API_CUPET,API_Avg,DIFF_API,...,CurrentRate,CurrentWC,WaterRate,CumOil,MaxOil,BeforeFrac,Normalised,FracSize,BeforeFracProd,AfterFracProd
0,P-103,753,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.000000,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
1,P-103,754,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.000000,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
2,P-103,755,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.000000,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
3,P-103,756,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.000000,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
4,P-103,757,1,1,andesitic tuff fine-grained,K2cp,17.0,NaN,17.000000,NaN,...,0.18,80.85,0.76,181022.0,283.0,49.0,86582.0,8.0,148.2,1444.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11956,B-4,1343,1,0,andesitic tuff coarse-grained,K1al,NaN,43.191358,43.191358,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11957,B-4,1344,1,0,andesitic tuff coarse-grained,K1al,NaN,43.191358,43.191358,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11958,B-4,1345,1,0,andesitic tuff coarse-grained,K1al,NaN,43.191358,43.191358,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11959,B-4,1346,1,0,andesitic tuff coarse-grained,K1al,NaN,43.191358,43.191358,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data = data.drop(columns=['API_PVT', 'API_CUPET', 'DIFF_API', 'GasGravity', 'OilGravity', 'P_Sat', 'Uo', 'Ug', 'Bo', 'Bg', 'CO2', 'H2S', 'InitialRate', 'CurrentRate', 'CurrentWC', 'WaterRate', 'MaxOil', 'BeforeFrac', 'Normalised', 'FracSize', 'BeforeFracProd', 'AfterFracProd'])
data

,Well,MD,Perfed,Fracced,Lithology,Age,API_Avg,CumOil
0,P-103,753,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
1,P-103,754,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
2,P-103,755,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
3,P-103,756,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
4,P-103,757,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
...,...,...,...,...,...,...,...,...
11956,B-4,1343,1,0,andesitic tuff coarse-grained,K1al,43.191358,NaN
11957,B-4,1344,1,0,andesitic tuff coarse-grained,K1al,43.191358,NaN
11958,B-4,1345,1,0,andesitic tuff coarse-grained,K1al,43.191358,NaN
11959,B-4,1346,1,0,andesitic tuff coarse-grained,K1al,43.191358,NaN


In [7]:
data = data.dropna()
data

,Well,MD,Perfed,Fracced,Lithology,Age,API_Avg,CumOil
0,P-103,753,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
1,P-103,754,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
2,P-103,755,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
3,P-103,756,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
4,P-103,757,1,1,andesitic tuff fine-grained,K2cp,17.000000,181022.0
...,...,...,...,...,...,...,...,...
11365,P-60,1019,1,0,polylithic cong,P21,26.398363,72560.0
11366,P-60,1020,1,0,polylithic cong,P21,26.398363,72560.0
11367,P-60,1021,1,0,polylithic cong,P21,26.398363,72560.0
11368,P-60,1022,1,0,polylithic cong,P21,26.398363,72560.0


In [8]:
target = data["CumOil"].values.reshape(-1,1)
print(target)
target

[[181022.]
 [181022.]
 [181022.]
 ...
 [ 72560.]
 [ 72560.]
 [ 72560.]]


array([[181022.],
       [181022.],
       [181022.],
       ...,
       [ 72560.],
       [ 72560.],
       [ 72560.]])

In [9]:
data = data.drop(columns=['CumOil','Well'])
data

,MD,Perfed,Fracced,Lithology,Age,API_Avg
0,753,1,1,andesitic tuff fine-grained,K2cp,17.000000
1,754,1,1,andesitic tuff fine-grained,K2cp,17.000000
2,755,1,1,andesitic tuff fine-grained,K2cp,17.000000
3,756,1,1,andesitic tuff fine-grained,K2cp,17.000000
4,757,1,1,andesitic tuff fine-grained,K2cp,17.000000
...,...,...,...,...,...,...
11365,1019,1,0,polylithic cong,P21,26.398363
11366,1020,1,0,polylithic cong,P21,26.398363
11367,1021,1,0,polylithic cong,P21,26.398363
11368,1022,1,0,polylithic cong,P21,26.398363


In [10]:
data_binary_encoded = pd.get_dummies(data, columns=["Lithology","Age"])
data_binary_encoded

,MD,Perfed,Fracced,API_Avg,Lithology_andesite,Lithology_andesitic tuff coarse-grained,Lithology_andesitic tuff fine-grained,Lithology_andesitic tuff med-grained,Lithology_andestic tuff,Lithology_argillaceous tuff,...,Lithology_tuffaceous cong,Lithology_tuffaceous lst,Age_K1al-K2cm,Age_K1al-K2tu,Age_K1ap-al,Age_K2cp,Age_K2upmaas,Age_P21,Age_P22,Age_nan
0,753,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,754,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,755,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,756,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,757,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11365,1019,1,0,26.398363,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
11366,1020,1,0,26.398363,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
11367,1021,1,0,26.398363,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
11368,1022,1,0,26.398363,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
data_binary_encoded.columns

Index(['MD', 'Perfed', 'Fracced', 'API_Avg', 'Lithology_andesite',
       'Lithology_andesitic tuff coarse-grained',
       'Lithology_andesitic tuff fine-grained',
       'Lithology_andesitic tuff med-grained', 'Lithology_andestic tuff',
       'Lithology_argillaceous tuff', 'Lithology_basalt',
       'Lithology_basaltic tuff', 'Lithology_breccia',
       'Lithology_claystone/marl', 'Lithology_claystone/marl/marl',
       'Lithology_limestone', 'Lithology_nan', 'Lithology_polylithic cong',
       'Lithology_polylithic sst', 'Lithology_tuffaceous cong',
       'Lithology_tuffaceous lst', 'Age_K1al-K2cm', 'Age_K1al-K2tu',
       'Age_K1ap-al', 'Age_K2cp', 'Age_K2upmaas', 'Age_P21', 'Age_P22',
       'Age_nan'],
      dtype='object')

In [12]:
data_binary_encoded = data_binary_encoded.drop(columns=['Age_nan', 'Lithology_nan', 'Age_P22', 'Lithology_limestone'])

In [13]:
data_binary_encoded

,MD,Perfed,Fracced,API_Avg,Lithology_andesite,Lithology_andesitic tuff coarse-grained,Lithology_andesitic tuff fine-grained,Lithology_andesitic tuff med-grained,Lithology_andestic tuff,Lithology_argillaceous tuff,...,Lithology_polylithic cong,Lithology_polylithic sst,Lithology_tuffaceous cong,Lithology_tuffaceous lst,Age_K1al-K2cm,Age_K1al-K2tu,Age_K1ap-al,Age_K2cp,Age_K2upmaas,Age_P21
0,753,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,754,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,755,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,756,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,757,1,1,17.000000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11365,1019,1,0,26.398363,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
11366,1020,1,0,26.398363,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
11367,1021,1,0,26.398363,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
11368,1022,1,0,26.398363,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [14]:
# Create a random forest classifier
rf = RandomForestRegressor(n_estimators=200)
rf = rf.fit(data_binary_encoded, target)
rf.score(data_binary_encoded, target)

0.9997071741804155

In [15]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([3.17511638e-01, 2.83365503e-06, 1.16637420e-01, 4.19695176e-01,
       6.31803546e-03, 1.82084151e-02, 4.84524956e-03, 1.11492223e-02,
       1.44246315e-02, 0.00000000e+00, 5.93591028e-03, 2.17132841e-06,
       8.56852524e-07, 4.07912821e-05, 0.00000000e+00, 2.74355970e-03,
       1.66551387e-05, 7.42230255e-03, 1.20996102e-03, 1.09528359e-02,
       9.93934041e-04, 5.39477774e-02, 7.15231897e-03, 6.80251362e-06,
       7.81501908e-04])

In [16]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, data_binary_encoded.feature_names), reverse=True)

AttributeError: 'DataFrame' object has no attribute 'feature_names'

In [ ]:
# Create a random forest classifier
# rf = RandomForestClassifier(n_estimators=200)
# rf = rf.fit(iris.data, iris.target)
# rf.score(iris.data, iris.target)